In [ ]:
import json
import csv
import pandas as pd
from pandas import DataFrame, Series
import dateutil.parser
from dateutil.tz import gettz
import datetime
import pytz
from functools import reduce

In [ ]:
tone_names = ['Sadness', 'Analytical', 'Joy', 'Fear', 'Tentative', 'Anger', 'Confident']
tone_names

In [ ]:
#keylogger DF

def roundTime(dt):
    dt = dt - datetime.timedelta(minutes=dt.minute, seconds=dt.second, microseconds=dt.microsecond)
    return dt

with open('../keylogger/logs/log_new.json', 'r') as f:
    keyloggerData = json.load(f)

tzinfos = { "EDT" : gettz("America/New_York") }

def extract_keyloggerData(data):
    results = []

    for d in data:
        result = [0]*(len(tone_names) + 7)
        try:
            result[7] = d['word_count']
            result[8] = d['uniqueword_count']
            result[9] = d['char_count']
            result[10] = d['backspace_count']
            result[11] = d['avg_dwelltime']
            result[12] = d['avg_flighttime']

            tones = d['document_tone']['tones']
            for i in range(len(tones)):
                score = tones[i]['score']
                tone_name = tones[i]['tone_name']
                tone_index = tone_names.index(tone_name)
                result[tone_index] = score            

            time = dateutil.parser.parse(d['time'] + " EDT", tzinfos=tzinfos)
            result[-1] = time
            results.append(tuple(result))
        except:
            continue
            
    return results

keyloggerDF = DataFrame(extract_keyloggerData(keyloggerData),
                        columns=[tone_name+"_score" for tone_name in tone_names] + ['word_count','uniqueword_count','char_count','backspace_count','avg_dwelltime','avg_flighttime','time'])
#keyloggerDF.time = keyloggerDF.time.apply(roundTime)
#keyloggerDF.time.apply(roundTime)
keyloggerDF

In [ ]:
#affectiva DF

local_tz = gettz('America/New_York')

def utc_to_local(utc_dt):
    return utc_dt.replace(tzinfo=local_tz)

with open('../affectiva/analyses/merged_file.json', 'r') as f:
    affectivaData = json.load(f)

for x in range(0, len(affectivaData)):
    test = datetime.datetime.fromtimestamp((affectivaData[x]['time']/ 1e3))
    affectivaData[x]['time'] = utc_to_local(datetime.datetime.fromtimestamp((affectivaData[x]['time']/ 1e3)))

affectivaDF = DataFrame(affectivaData)
# affectivaDF.time = affectivaDF.time.apply(roundTime)
# affectivaDF.time.apply(roundTime)
#affectivaDF.emoji = affectivaDF.emoji.apply(lambda x: ",".join(x))
del affectivaDF['emoji']
#affectivaDF.emotions = affectivaDF.emotions.apply(lambda x: ",".join(x))
del affectivaDF['emotions']
del affectivaDF['max_attention']
del affectivaDF['min_attention']
del affectivaDF['max_engagement']
del affectivaDF['min_engagement']
del affectivaDF['max_valence']
del affectivaDF['min_valence']

affectivaDF

# mood reporter DF

responsesDF = pd.read_csv("../reporter/responses.tsv", sep='\t', header=0)

timeValues = responsesDF.time.values

for x in range(0, len(timeValues)):
    timeValues[x] = dateutil.parser.parse(timeValues[x] + " EDT", tzinfos=tzinfos)

# responsesData.time = timeValues
#responsesData.time = responsesData.time.apply(roundTime)
#responsesData.time.apply(roundTime)

activity_names = list(set(list(map(lambda x: x.lower().strip(), reduce(lambda x,y: x+y, [x.split(",") for x in list(responsesDF.activity.values)])))))

for activity_name in activity_names:
    responsesDF[activity_name.replace(" ", "_") + "_activity"] = responsesDF.activity.apply(lambda x: activity_name in x.lower())

location_names = list(set(list(map(lambda x: x.lower().strip(), reduce(lambda x,y: x+y, [x.split(",") for x in list(responsesDF.location.values)])))))

def split_locations(locations):
    return list(map(lambda x: x.lower().strip(), locations.split(",")))

for location_name in location_names:
    responsesDF[location_name.replace(" ", "_") + "_location"] = responsesDF.location.apply(lambda x: location_name in split_locations(x))
    
del responsesDF['moodNotes']
del responsesDF['trigger']
del responsesDF['activity']
del responsesDF['location']

responsesDF

In [ ]:
#productivity DF

with open('../getAPIdata/productivity.json', 'r') as f:
    productivityFile = json.load(f)

productivityData = productivityFile['rows']

final_productivityData = [];

for x in range(0, len(productivityData)):
    if(productivityData[x][0] > '2018-04-01T90:00:00'):
        time = dateutil.parser.parse(productivityData[x][0] + " EDT", tzinfos=tzinfos)
        prod_score = productivityData[x][4]
        final_productivityData.append((time, prod_score))

productivityDF = DataFrame(final_productivityData, columns=['time', 'productivity_score'])
#productivityDF.time = productivityDF.time.apply(roundTime)
#productivityDF.time.apply(roundTime)

productivityDF

In [ ]:
exerciseDF = pd.read_csv("../S_Health/com.samsung.health.exercise.201808072208.csv", header=0)
exerciseTimeVals = exerciseDF.create_time.values

for x in range(0, len(exerciseTimeVals)):
    exerciseTimeVals[x] = dateutil.parser.parse(exerciseTimeVals[x] + " EDT", tzinfos=tzinfos)

exerciseDF.create_time = exerciseTimeVals
exerciseDF = exerciseDF[['create_time', 'distance', 'mean_speed', 'calorie']]
exerciseDF.columns = ['time', 'distance', 'mean_speed', 'calorie']
exerciseDF.sort_values('time', inplace=True)
exerciseDF = exerciseDF.reset_index(drop=True)
exerciseDF = exerciseDF[(exerciseDF['time'] > '2018-05-31')]

exerciseDF

In [ ]:
o2SatDF = pd.read_csv("../S_Health/com.samsung.health.oxygen_saturation.201808072208.csv", header=0)
o2TimeVals = o2SatDF.create_time.values
#o2TimeZoneVals = o2SatDF.time_offset.values

for x in range(0, len(o2TimeVals)):
    o2TimeVals[x] = dateutil.parser.parse(o2TimeVals[x] + " EDT", tzinfos=tzinfos)

o2SatDF.create_time = o2TimeVals
o2SatDF = o2SatDF[['create_time', 'heart_rate', 'spo2']]
o2SatDF.columns = ['time', 'heart_rate', 'spo2']
o2SatDF.sort_values('time', inplace=True)
o2SatDF = o2SatDF.reset_index(drop=True)
o2SatDF

In [ ]:
#tabcounter

with open('../getAPIdata/chromeactivity.json', 'r') as f:
    tabCounterData = json.load(f)

newTabData = []

for key in tabCounterData:
    time = utc_to_local(datetime.datetime.fromtimestamp((int(key)/ 1e3)))
    current_tabCount = tabCounterData[key]['current_tabCount']
    current_windowCount = tabCounterData[key]['current_windowCount']
    tabs_activated = tabCounterData[key]['tabs_activated']
    tabs_created = tabCounterData[key]['tabs_created']
    windows_created = tabCounterData[key]['windows_created']

    newTabData.append([time, current_tabCount, current_windowCount,tabs_activated,tabs_created,windows_created])

tabColumns = ['time', 'current_tabCount', 'current_windowCount','tabs_activated','tabs_created','windows_created']
tabCounterDF = DataFrame(newTabData, columns=tabColumns)
tabCounterDF

In [ ]:
qualityTimeDF = pd.read_csv("../QualityTime/1533931014602_ENTIRE_HISTORY_1470859008205_1533931008205.csv", header=0)
qtTimeVals = qualityTimeDF.End_Time.values

newQTtimeVals = []
for x in range(0, len(qtTimeVals)):
    time = utc_to_local(datetime.datetime.fromtimestamp((int(qtTimeVals[x])/ 1e3)))
    newQTtimeVals.append(time)


usageVals = qualityTimeDF.Usage.values
newQTusageVals = []

for x in range(0, len(usageVals)):
    string = usageVals[x]
    minSubstring = "min"
    hrSubstring = "hr"
    finalSecs = 0
    
    if hrSubstring in string:
        removeHr = string[0:1]
        newHr = int(removeHr) * 3600
        finalSecs += newHr
    elif minSubstring in string:
        removeMin = string[-6:-4]
        newMin = int(removeMin) * 60
        finalSecs += newMin
    else:
        removeSec = string[0:-4]
        newSec = int(removeSec)
        finalSecs += newSec
        
    newQTusageVals.append(finalSecs)

    
qualityTimeDF.End_Time = newQTtimeVals
qualityTimeDF.Usage = newQTusageVals
qualityTimeDF = qualityTimeDF[['End_Time', 'Usage']]
qualityTimeDF.columns = ['time', 'usage_in_sec']

compressed = []
current_hour = qualityTimeDF['time'][0].replace(microsecond=0,second=0,minute=0)
total_usage = 0
unlocks = 0

for index, row in qualityTimeDF.iterrows():
    if (index == len(qualityTimeDF) -1):
        holder = [0] * 3
        holder[0] = current_hour
        holder[1] = unlocks
        holder[2] = total_usage
        compressed.append(tuple(holder))
    elif (row['time'].replace(microsecond=0,second=0,minute=0) == current_hour):
        total_usage += row['usage_in_sec']
        unlocks +=1
    else:
        holder = [0] * 3
        holder[0] = current_hour
        holder[1] = unlocks
        holder[2] = total_usage
        compressed.append(tuple(holder))
        total_usage = 0
        unlocks = 0
        current_hour = row['time'].replace(microsecond=0,second=0,minute=0)
        total_usage += row['usage_in_sec']
        unlocks +=1
        

newQTDF = DataFrame(compressed, columns = ['time', 'unlocks', 'total_usage_in_secs'])
newQTDF

In [ ]:
#QTDF
responsesTimeVals = responsesDF.loc[:,'time']
responsesTimeVals = responsesTimeVals.apply(roundTime)
responsesTimeValsDF = DataFrame(responsesTimeVals)

QTframes = [newresponsesTimeValsDF, newQTDF]
QTresult = pd.concat(QTframes)
QTresult.sort_values('time', inplace=True)
QTresult = QTresult.reset_index(drop=True)
QTresult = QTresult.fillna(0)

QTresult

In [ ]:
# merge all the data

mergedData = []

all_columns = list(responsesDF.columns) + list(keyloggerDF.columns.drop('time')) + list(affectivaDF.columns.drop('time')) + list(productivityDF.columns.drop('time')) + list(o2SatDF.columns.drop('time')) + list(tabCounterDF.columns.drop('time')) + list(newQTDF.columns.drop('time'))

def process_row(row):
    current_time = row['time']

    output_values = list(row.values)

    for other_df in [keyloggerDF, affectivaDF, productivityDF, o2SatDF, tabCounterDF, newQTDF]:
        candidates = other_df[(current_time > other_df['time']) &
                              (current_time - other_df['time'] < datetime.timedelta(hours=5))]
        if candidates.empty:
            return None
        else:
            index_of_max = candidates['time'].argmax()
            candidate = candidates.ix[index_of_max].drop('time')
            output_values += list(candidate.values)
    return output_values

for index, row in responsesDF.iterrows():
    processed_row = process_row(row)
    if processed_row is not None:
        mergedData.append(processed_row)

mergedDF = DataFrame(mergedData, columns = all_columns)
mergedDF

In [ ]:
# mergedDF_copy = mergedDF[['time']]
mergedDFtime = mergedDF.loc[:,'time']
mergedDF_copy = DataFrame(mergedDFtime)


mergedDF_copy.time.apply(roundTime)
#mergedTimeVals = mergedDF_copy[['time']]

newExerciseDF = exerciseDF[(exerciseDF['time'] > '2018-05-18')]
newExerciseDF.sort_values('time', inplace=True)
newExerciseDF = newExerciseDF.reset_index(drop=True)

#exerciseDFTimeVals = newExerciseDF[['time']]

#for x in range(0, len(newExerciseDF)):
    

frames = [mergedDF_copy, newExerciseDF]
result = pd.concat(frames)
result.sort_values('time', inplace=True)
result = result.reset_index(drop=True)
result = result.fillna(0)


columns = list(mergedDF.columns) + list(result.columns.drop('time'))

def process_row(row):
    current_time = row['time']

    output_values = list(row.values)

    #for other_df in result:
    candidates = result[(current_time > result['time']) &
                          (current_time - result['time'] < datetime.timedelta(hours=2))]
    if candidates.empty:
#         return None
        output_values += [0,0,0]
    else:
        index_of_max = candidates['time'].argmax()
        candidate = candidates.ix[index_of_max].drop('time')
        
        if(candidate['distance'] > 0):
            output_values += list(candidate.values)
        else:
            index_of_min = candidates['time'].argmin()
            candidate = candidates.ix[index_of_min].drop('time')
            output_values += list(candidate.values)

    return output_values

newMergedData = []
for index, row in mergedDF.iterrows():
    processed_row = process_row(row)
    if processed_row is not None:
        newMergedData.append(processed_row)

newMergedDF = DataFrame(newMergedData, columns = columns)
newMergedDF


In [ ]:
newMergedDF.fillna(0)

In [ ]:
newMergedDF.to_csv("mergedDF.csv", sep='\t', encoding='utf-8')

In [ ]:
corr = newMergedDF.corr().fillna(0)
# corr.to_csv("correlationmatrix.csv", sep='\t', encoding='utf-8')

In [ ]:
import matplotlib
import matplotlib.pyplot as plt
pd.plotting.scatter_matrix(newMergedDF, figsize=(88, 88))
# plt.show()

In [ ]:
# plt.figure(figsize=(88,88))
# plt.matshow(mergedDF.corr())
# plt.xticks(range(len(mergedDF.columns)), mergedDF.columns)
# plt.yticks(range(len(mergedDF.columns)), mergedDF.columns)
# plt.colorbar()
# plt.show()

In [ ]:
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

cmap = cmap=sns.diverging_palette(5, 250, as_cmap=True)

def magnify():
    return [dict(selector="th",
                 props=[("font-size", "7pt")]),
            dict(selector="td",
                 props=[('padding', "0em 0em")]),
            dict(selector="th:hover",
                 props=[("font-size", "12pt")]),
            dict(selector="tr:hover td:hover",
                 props=[('max-width', '200px'),
                        ('font-size', '12pt')])
]

corr.style.background_gradient(cmap, axis=1)\
    .set_properties(**{'max-width': '30px', 'min-width': '60px', 'min-height': '60px', 'max-height': '60px', 'font-size': '8pt', 'padding': '1em 1em'})\
    .set_caption("Hover to magify")\
    .set_precision(2)\
#     .set_table_styles(magnify())

In [ ]:
import seaborn as sns
%matplotlib inline
import numpy as np

# Generate a mask for the upper triangle
mask = np.zeros_like(corr, dtype=np.bool)
mask[np.triu_indices_from(mask)] = True

# Set up the matplotlib figure
f, ax = plt.subplots(figsize=(30, 25))

# ax.set_axis_bgcolor("white")

# Generate a custom diverging colormap
cmap = sns.diverging_palette(359, 230, s=88, l=74, sep=10, n=9, as_cmap=True)
sns.set_style("white")

# Draw the heatmap with the mask and correct aspect ratio
sns.heatmap(corr, cmap=cmap, vmax=.3, center=0,
            square=True, linewidths=0, cbar_kws={"shrink": 0.3})

In [ ]:
import seaborn as sns
%matplotlib inline

sns.choose_diverging_palette()

In [ ]:
newMergedDF.corr().mood.sort_values(ascending=False)

In [ ]:
mergedDF.columns
featuresList = ['errands_activity', 'resting_activity',
       'schoolwork_activity', 'therapy_activity', 'school_work_activity',
       'just_woke_up_activity', 'ash_stuff_activity', 'workshop_activity',
       'leisure_activity', 'eating_activity', 'email_activity',
       'with_k_activity', 'reading_activity', 'commuting_activity',
       'chatting_activity', 'with_friends_activity', 'in_class_activity',
       'kevin_home_location', 'school_location', 'library_location',
       'home_location', 'therapy_location', 'Sadness_score',
       'Analytical_score', 'Joy_score', 'Fear_score', 'Tentative_score',
       'Anger_score', 'Confident_score', 'blinks', 'max_attention',
       'max_engagement', 'max_valence', 'min_attention', 'min_engagement',
       'min_valence', 'productivity_score']

In [ ]:
from sklearn import linear_model
y = mergedDF["mood"]
X = mergedDF[featuresList]
myLinearModel = linear_model.LinearRegression()
myLinearModel = myLinearModel.fit(X, y)

In [ ]:
myLinearModel.predict(X)

In [ ]:
y.values

In [ ]:
import numpy as np
np.set_printoptions(suppress=True)
list(zip(featuresList,np.round(model.coef_, 3)))

In [ ]:
from sklearn import tree
y = mergedDF["mood"]
X = mergedDF[featuresList]
decision_tree = tree.DecisionTreeRegressor()
decision_tree = decision_tree.fit(X, y)

In [ ]:
decision_tree.feature_importances_
list(zip(featuresList,np.round(model.feature_importances_, 3)))

In [ ]:
X.iloc[0].values

In [ ]:
print("\n".join(str(x)+", # " +y for x,y in zip(X.iloc[0].values,featuresList)))

In [ ]:
fakehappyX = [False, # errands_activity
True, # resting_activity
False, # schoolwork_activity
True, # therapy_activity
False, # school_work_activity
False, # just_woke_up_activity
False, # ash_stuff_activity
False, # workshop_activity
True, # leisure_activity
True, # eating_activity
False, # email_activity
True, # with_k_activity
False, # reading_activity
False, # commuting_activity
True, # chatting_activity
False, # with_friends_activity
False, # in_class_activity
True, # kevin_home_location
False, # school_location
False, # library_location
False, # home_location
False, # therapy_location
0.0, # Sadness_score
0.0, # Analytical_score
99.0, # Joy_score
0.0, # Fear_score
0.0, # Tentative_score
0.0, # Anger_score
0.0, # Confident_score
2, # blinks
98.7772064209, # max_attention
90.76793289185, # max_engagement
90.0, # max_valence
95.8359222412, # min_attention
90.0814501345158, # min_engagement
20.81201314926, # min_valence
90.14] # productivity_score

In [ ]:
model.predict([fakehappyX])

In [ ]:
fakeSadX = [True, # errands_activity
False, # resting_activity
True, # schoolwork_activity
False, # therapy_activity
True, # school_work_activity
False, # just_woke_up_activity
True, # ash_stuff_activity
False, # workshop_activity
False, # leisure_activity
False, # eating_activity
False, # email_activity
False, # with_k_activity
False, # reading_activity
True, # commuting_activity
False, # chatting_activity
False, # with_friends_activity
False, # in_class_activity
False, # kevin_home_location
True, # school_location
False, # library_location
False, # home_location
False, # therapy_location
99.0, # Sadness_score
0.0, # Analytical_score
0.0, # Joy_score
70.0, # Fear_score
40.0, # Tentative_score
60.0, # Anger_score
0.0, # Confident_score
20, # blinks
70.7772064209, # max_attention
20.76793289185, # max_engagement
-40.0, # max_valence
65.8359222412, # min_attention
10.0814501345158, # min_engagement
-90.81201314926, # min_valence
40.14] # productivity_score

In [ ]:
model.predict([fakeSadX])

In [ ]:
myLinearModel.predict([fakehappyX])

In [1]:
from sklearn.model_selection import train_test_split

#set aside 20% of the data to use as test data; the rest will be used as training date
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.20, random_state=42)

#train a linear model on the training data
myLinearModel = linear_model.LinearRegression()
myLinearModel = myLinearModel.fit(X_train, y_train)

#predict on test data and print the results
y_pred_lm = myLinearModel.predict(X_test)
y_pred_lm

NameError: name 'X' is not defined

In [ ]:
#calculate the mean absolute error
from sklearn.metrics import mean_squared_error, mean_absolute_error
mean_absolute_error(y_test, y_pred_lm)

In [ ]:
#train a decision tree on the training data
decision_tree = tree.DecisionTreeRegressor()
decision_tree = decision_tree.fit(X_train, y_train)

#predict on test data and print the results
y_pred_dt = decision_tree.predict(X_test)
y_pred_dt

In [ ]:
#calculate the decision tree's mean absolute error
from sklearn.metrics import mean_squared_error, mean_absolute_error
mean_absolute_error(y_test, y_pred_dt)

In [ ]:
#train a random forest on the training data
from sklearn import ensemble
random_forest = ensemble.RandomForestRegressor()
random_forest = random_forest.fit(X_train, y_train)

#predict on the test data and print the results
y_pred_rf = random_forest.predict(X_test)
y_pred_rf

In [ ]:
#calculate the random forest's error
mean_absolute_error(y_test, y_pred_rf)

In [ ]:
print("Test values, Random Forest Predictions, Decision Tree Predictions, Linear Model Predictions")
list(zip(y_test, y_pred_rf, y_pred_dt, y_pred_lm))

In [ ]:
predictionsDF = DataFrame(list(zip(y_test, y_pred_rf, y_pred_dt, y_pred_lm)), columns=["Test values", "Random Forest Predictions", "Decision Tree Predictions", "Linear Model Predictions"])
predictionsDF
